In [81]:
import pandas as pd
import tensorflow 
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
import re
import tqdm
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
data=pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv')
column_names = ["tweet_id","about","text", "sentiment"]
data.columns=column_names

In [ ]:
test_data=data=pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv')
column_names_test = ["tweet_id","about","text"]
data.columns=column_names_test

In [40]:
data.isna().sum()

tweet_id       0
about          0
text           0
sentiment    686
dtype: int64

In [84]:
len(data)

74681

In [ ]:
def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return clean_txt
data['processed_text'] = data['text'].apply(process_text)

100%|██████████| 1/1 [00:00<00:00, 9341.43it/s]


In [38]:
plt.rcParams['font.family'] = 'DejaVu Sans'
sentiment_counts = data['sentiment'].value_counts()
plt.figure(figsize=(8, 6))
plt.barh(sentiment_counts.index, sentiment_counts.values, color=['green', 'red', 'blue'])
plt.xlabel("Frequency")
plt.ylabel("Sentiment")
plt.title("Sentiment Frequencies")
plt.show()

/opt/conda/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129310 (\N{HAND WITH INDEX AND MIDDLE FINGERS CROSSED}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129392 (\N{SMILING FACE WITH SMILING EYES AND THREE HEARTS}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129299 (\N{NERD FACE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129397 (\N{OVERHEATED FACE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.10/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129335 (\N{SHRUG}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/l

ValueError: 
people have mad respect for the Hispanic men will stand outside the gas station/Home front/ Lowe’s in the cold, rain l, whatever months.. only to sell $$ Hustle.
                                                                                                                                                       ^
ParseException: Expected end of text, found '$'  (at char 151), (line:1, col:152)

<Figure size 800x600 with 1 Axes>

In [ ]:
data.isna().sum()
data.dropna(axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
X_train=data[['aprocessed_text']]
y_train=data['sentiment']
tokenizer = Tokenizer(oov_token='<OOV>') #may there is words not seen in x_train so model will not put to it sequance umber in testing
tokenizer.fit_on_texts(rev)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
max_sequence_length = max([len(seq) for seq in X_train_sequences])
X_train_padded=pad_sequences(X_train_sequences,maxlen=max_sequence_length)
X_test_padded=pad_sequences(X_test_sequences,maxlen=max_sequence_length)

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>')
model=tf.keras.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64,input_length=max_sequence_length),
    Conv1D(64,3,activation='relu'),
    Conv1D(128,7,activation='relu'),
    GlobalMaxPooling1D())
    Conv1D(64,15,activation='relu'),
    Dense(units=3, activation='softmax')   
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_padded, y_train, epochs=3, batch_size=128)
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)